In [14]:
import copy
from qiskit.providers.aer.pulse.hamiltonian_model import HamiltonianModel
from qiskit.providers.aer.pulse.qobj.opparse import HamiltonianParser
from qutip import sigmax
from qutip.control.grape import _overlap
from notebooks.qutip_wrapper import qutip_optimize_wrapper, qutip_simulate_wrapper
%matplotlib inline
# import matplotlib.pyplot as plt
import numpy as np
# import datetime
# from qutip_wrapper import qutip_optimize_wrapper, qutip_simulate_wrapper
from notebooks.qiskit_scripts import *
# from qutip import identity, sigmax, sigmaz, sigmay, hadamard_transform


In [4]:
import qiskit
qiskit.__qiskit_version__

{'qiskit-terra': '0.14.2',
 'qiskit-aer': '0.5.2',
 'qiskit-ignis': '0.3.3',
 'qiskit-ibmq-provider': '0.7.2',
 'qiskit-aqua': '0.7.3',
 'qiskit': '0.19.6'}

In [5]:
from qiskit import IBMQ, pulse, QuantumCircuit
from qiskit.pulse import Play
# import qiskit

IBMQ.load_account()

provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend('ibmq_armonk')
armonk_backend = backend
# original_backend = provider.get_backend('ibmq_armonk')

In [73]:
backend.defaults().instruction_schedule_map

TypeError: 'InstructionScheduleMap' object is not callable

In [6]:
provider = IBMQ.get_provider(hub='ibm-q-internal', group='deployed', project='default')
valencia_backend = provider.get_backend('ibmq_valencia')
# ham = backend.configuration().hamiltonian

In [65]:
config = valencia_backend.configuration()
subsystem_list = list(range(config.n_qubits))
ham_string = config.hamiltonian
valencia_hamiltonian = HamiltonianModel.from_dict(ham_string, subsystem_list)

In [53]:
config = backend.configuration()
subsystem_list = list(range(config.n_qubits))
ham_string = config.hamiltonian
hamiltonian = HamiltonianModel.from_dict(ham_string, subsystem_list)

In [66]:
print(ham_string['h_latex'])

\begin{align} \mathcal{H}/\hbar = & \sum_{i=0}^{4}\left(\frac{\omega_{q,i}}{2} (\mathbb{1}_i-\sigma_i^{z})+ \Omega_{d,i}D_i(t)\sigma_i^{X}\right) \\ & + J_{0,1}(\sigma_{0}^{+}\sigma_{1}^{-}+\sigma_{0}^{-}\sigma_{1}^{+}) + J_{1,3}(\sigma_{1}^{+}\sigma_{3}^{-}+\sigma_{1}^{-}\sigma_{3}^{+}) + J_{3,4}(\sigma_{3}^{+}\sigma_{4}^{-}+\sigma_{3}^{-}\sigma_{4}^{+}) + J_{1,2}(\sigma_{1}^{+}\sigma_{2}^{-}+\sigma_{1}^{-}\sigma_{2}^{+}) \\ & + \Omega_{d,0}(U_{0}^{(0,1)}(t))\sigma_{0}^{X} + \Omega_{d,1}(U_{1}^{(1,0)}(t)+U_{3}^{(1,3)}(t)+U_{2}^{(1,2)}(t))\sigma_{1}^{X} \\ & + \Omega_{d,2}(U_{4}^{(2,1)}(t))\sigma_{2}^{X} + \Omega_{d,3}(U_{5}^{(3,1)}(t)+U_{6}^{(3,4)}(t))\sigma_{3}^{X} \\ & + \Omega_{d,4}(U_{7}^{(4,3)}(t))\sigma_{4}^{X} \\ \end{align}


\begin{align} \mathcal{H}/\hbar = & \sum_{i=0}^{4}\left(\frac{\omega_{q,i}}{2} (\mathbb{1}_i-\sigma_i^{z})+ \Omega_{d,i}D_i(t)\sigma_i^{X}\right) \\ & + J_{0,1}(\sigma_{0}^{+}\sigma_{1}^{-}+\sigma_{0}^{-}\sigma_{1}^{+}) + J_{1,3}(\sigma_{1}^{+}\sigma_{3}^{-}+\sigma_{1}^{-}\sigma_{3}^{+}) + J_{3,4}(\sigma_{3}^{+}\sigma_{4}^{-}+\sigma_{3}^{-}\sigma_{4}^{+}) + J_{1,2}(\sigma_{1}^{+}\sigma_{2}^{-}+\sigma_{1}^{-}\sigma_{2}^{+}) \\ & + \Omega_{d,0}(U_{0}^{(0,1)}(t))\sigma_{0}^{X} + \Omega_{d,1}(U_{1}^{(1,0)}(t)+U_{3}^{(1,3)}(t)+U_{2}^{(1,2)}(t))\sigma_{1}^{X} \\ & + \Omega_{d,2}(U_{4}^{(2,1)}(t))\sigma_{2}^{X} + \Omega_{d,3}(U_{5}^{(3,1)}(t)+U_{6}^{(3,4)}(t))\sigma_{3}^{X} \\ & + \Omega_{d,4}(U_{7}^{(4,3)}(t))\sigma_{4}^{X} \\ \end{align}

In [63]:
$\begin{align} \mathcal{H}/\hbar = & \sum_{i=0}^{0}\left(\frac{\omega_{q,i}}{2} (\mathbb{1}_i-\sigma_i^{z})+ \Omega_{d,i}D_i(t)\sigma_i^{X}\right) \\ \end{align}$

SyntaxError: invalid syntax (<ipython-input-63-fc3e2bca6e3e>, line 1)

In [59]:
# ham_string
hamiltonian._system[1][0]
# valencia_hamiltonian._system[8]
# parser = HamiltonianParser(['h_str'], ham['qub'], ham['osc'])
# valencia_hamiltonian._system

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[0. 1.]
 [1. 0.]]

In [10]:
print(config.hamiltonian['h_latex'])

\begin{align} \mathcal{H}/\hbar = & \sum_{i=0}^{0}\left(\frac{\omega_{q,i}}{2} (\mathbb{1}_i-\sigma_i^{z})+ \Omega_{d,i}D_i(t)\sigma_i^{X}\right) \\ \end{align}


In [11]:
for field in valencia_hamiltonian._system:
    variable_name = field[1]
    sigma_term = field[0]
H_d = 0

In [12]:
H_d=hamiltonian._system[0]
for field in hamiltonian._system[1:]:
    variable_name = field[1]
    sigma_term = field[0]
    H_d += hamiltonian._variables[variable_name] * sigma_term

KeyError: 'omegad0*D0'

In [ ]:
hamiltonian._variables

In [ ]:
    from qutip import sigmax
    # sigma_term + sigmax()
# hamiltonian._variables
# hamiltonian._variables
hamiltonian._compute_drift_data()
hamiltonian._calculate_hamiltonian_channels()
hamiltonian._system[1:]

In [ ]:
# sigmax()
# sigma_term + 0
# variable_name
backend.defaults().instruction_schedule_map._map.keys()
backend.defaults().instruction_schedule_map._map['x']

In [ ]:

# from qiskit.providers.aer.pulse.syste_models.string_model_parser.string_model_parser import HamiltonianParser

In [50]:
config = backend.configuration()
subsystem_list = list(range(config.n_qubits))
ham = config.hamiltonian

#@TZ dimension of oscillator subspace? unclear and ham['osc'] ∅ {}
parser = HamiltonianParser(valencia_hamiltonian['h_str'], ham['qub'], ham['osc'])

TypeError: 'HamiltonianModel' object is not subscriptable

In [42]:
parser.parse([0])
parser.compiled


# parser.dim_osc


[(Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
  Qobj data =
  [[0. 0.]
   [0. 0.]],
  'wq0/2'),
 (Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
  Qobj data =
  [[1. 0.]
   [0. 1.]],
  'omegad0*D0')]